In [ ]:
# Apply transformation to set an image to each row in order to know it
df_as_single = pd.concat(df_yolo.apply(lambda row: row['objects'].assign(image=row['image']), axis=1).to_list())
df_as_single

In [ ]:
df_as_single['class'].hist()

In [ ]:
df_as_single['class'].mode()

In [ ]:
# Show class distribution
plot_sorted_counts(df_as_single, 'class')

In [ ]:
# We check the class that correspond to the value
class_number = df_as_single['class'].mode()[0]
# Find which category corresponds
[key for key, val in category_dict.items() if val == class_number]

In [ ]:
# Filter by list that contain this class
df_as_single_filtered = df_as_single[df_as_single['class'] == class_number]
df_as_single_filtered

In [ ]:
df_as_single_filtered.reset_index(drop=True, inplace=True)
df_as_single.reset_index(drop=True, inplace=True)
mask = df_as_single['image'].isin(df_as_single_filtered['image'])
filtered_rows = df_as_single[mask]
filtered_rows

In [ ]:
filtered_rows.loc[153:154]

In [ ]:
filtered_rows[filtered_rows['image'].astype(str) == "datasets/Mapillary Traffic Sign/images/yuc-hYWnqfRVfXUpJIAf9A.jpg"]

In [ ]:
filtered_rows_unique = filtered_rows.groupby('image').filter(lambda x: x['class'].nunique() == 1)
filtered_rows_unique

In [ ]:
rows_to_delete = filtered_rows_unique.drop_duplicates(subset='image')
rows_to_delete

In [ ]:
mask = df_as_single['image'].isin(rows_to_delete['image'])
filtered_df = df_as_single[~mask]
filtered_df

In [ ]:
# Show filtered rows
filtered_df['class'].hist()

In [ ]:
filtered_df['class'].value_counts()

In [ ]:
# Show class distribution
plot_sorted_counts(filtered_df, 'class')

# Here we can confirm that stratification is not a valid option because most of "other sign" classes are tight-coupled with other classes

https://www.hindawi.com/journals/mpe/2022/9571513/

In [ ]:
# Get rows where the 'class' column equals class_number
rows_to_delete_2 = filtered_df[filtered_df['class'] == class_number]

# Then get all rows whose 'image' is in the 'image' column of class_rows
mask = filtered_df['image'].isin(rows_to_delete_2['image'])
filtered_df_2 = filtered_df[~mask]
filtered_df_2

In [ ]:
# Show filtered rows excluding other sign
filtered_df_2['class'].hist()

In [ ]:
filtered_df_2['image'].unique()

In [ ]:
filtered_df_2['class'].value_counts()

In [ ]:
# First get the unique image IDs from dataframe `filtered_df_2`
unique_image_ids = filtered_df_2['image'].unique()

# Then, from `df_yolo`, select only the rows where image_id is in `unique_image_ids`
yolo_filtered_df_2 = df_yolo[df_yolo['image'].isin(unique_image_ids)].copy()
yolo_filtered_df_2

In [ ]:
# Show class distribution
plot_sorted_counts(filtered_df_2, 'class')

# We see that augmentation is the best here

In [ ]:
# This filtered dataset is the number of rows
df_as_single_filtered['class'].value_counts()

In [ ]:
filtered_df_2['class'].value_counts().max()

In [ ]:
# Now we can add the missing class label to last dataset which is reduced
#temp = list_single_classes.groupby('image')['class'].unique()

In [ ]:
# Some classes can have empty datasets
df_yolo[df_yolo['objects'].str.len() == 0]

In [ ]:
# Group by class and count unique image ids for each class
class_image_counts = filtered_df_2.groupby('class')['image'].nunique()

# Find the classes that do not have at least one image id with a single label
classes_without_unique_images = class_image_counts[class_image_counts < 1]

if len(classes_without_unique_images) > 0:
    print("Following classes do not have at least one unique image instance:")
    print(classes_without_unique_images.index.tolist())
else:
    print("All classes have at least one unique image instance.")

In [ ]:
final_count = 0
new_rows = []
new_objects_rows = []

df_yolo_copy = df_yolo.copy()
list_single_classes_copy = filtered_df_2.copy()

row_count = len(df_yolo_copy)
counts_per_class = list_single_classes_copy['class'].value_counts()
total_classes = len(counts_per_class.index)
sum_of_counts = counts_per_class.sum()

mean_count = sum_of_counts / total_classes
max_count = counts_per_class.max()

while mean_count < max_count:
    idx = np.random.randint(row_count)
    row = df_yolo_copy.iloc[idx]
    inner_df = row['objects']
    
    pass_class = False
    
    if not inner_df.empty:
        unique_classes = inner_df['class'].unique()
        
        for class_label in unique_classes:
            if counts_per_class[class_label] >= max_count:
                pass_class = True
                break
        
        if not pass_class:
            final_count += 1
            new_rows.append(row)
            new_objects_rows.extend(inner_df.assign(image=row['image']).to_dict('records'))
            
            counts_per_class = counts_per_class.add(inner_df['class'].value_counts(), fill_value=0)
            total_classes = len(counts_per_class.index)
            sum_of_counts = counts_per_class.sum()

            mean_count = sum_of_counts / total_classes
            max_count = counts_per_class.max()

            # plot_sorted_counts(list_single_classes_copy, 'class')
            print(f"Average is {mean_count} / {max_count} and total is {sum_of_counts}")

df_yolo_copy = df_yolo_copy.append(new_rows)
list_single_classes_copy = list_single_classes_copy.append(pd.DataFrame(new_objects_rows))

# Process df_final as needed
print(f"Total is {final_count}")
print(f"Average is {mean_count} / {max_count}")

In [ ]:
# This gets how many for each instace
# class df_as_single['class'].value_counts().loc[n]
df_as_single['class'].value_counts().max()
#